In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets
from scipy.optimize import fmin_tnc


In [137]:
iris = datasets.load_iris()

data = pd.DataFrame(iris['data'])
target = pd.DataFrame(iris['target'])
dt = pd.concat([data, target], axis =1)
dt.columns = ['sl', 'sw', 'pl', 'pw', 'type']
dt = dt.loc[dt['type'] != 0]

X1 = dt.drop('type', axis = 1)
Y1 = dt['type'] - 1

X1 = np.c_[np.ones((X1.shape[0], 1)), X1]
Y1 = Y1[:, np.newaxis]

In [16]:
def predict(theta, X):
    return 1 / (1 + np.exp(- np.dot(X, theta)))

In [143]:
lr = 0.05
max_epoch = 1000
gamma = 0.9
eps = 0.00000001

theta = np.random.normal(size=(X1.shape[1]))
theta1 = theta.copy()
theta2 = theta.copy()

gradient_loss = []

velocity_pred = np.zeros(X1.shape[1])
velocity = np.zeros(X1.shape[1])
nesterov_loss = []

e_sq_grad = np.zeros(X1.shape[1])
gradient = np.zeros(X1.shape[1])
rmsp_loss = []


*** SGD

In [144]:
for _ in range(max_epoch):
    sigm1 = predict(theta, X1)
    theta -= lr * np.sum(np.dot((sigm1 - Y1.flatten()), X1))/len(sigm1)
    loss1 = - np.mean(np.log(sigm1) * Y1 + np.log(1 - sigm1) * (1 - Y1))
#    print(theta1)
    gradient_loss.append(loss1)
print(theta)

[ 0.3036847   1.02279311 -0.98838147 -0.60160891 -1.91692144]


*** Nesterov

In [145]:
for _ in range(max_epoch):
    sigm1 = predict(theta1, X1)
    loss1 = - np.mean(np.log(sigm1) * Y1 + np.log(1 - sigm1) * (1 - Y1))
    nesterov_loss.append(loss1)
    sigm1 = predict(theta1 - gamma * velocity_pred, X1)
    velocity = (gamma * velocity_pred + lr * np.sum(np.dot((sigm1 - Y1.flatten()), X1))) / len(sigm1)
    theta1 -= velocity
#    print(theta1)
    velocity_pred = velocity

print(theta1)

[ 0.61877679  1.3378852  -0.67328938 -0.28651683 -1.60182936]


*** rmsprop

In [146]:
for _ in range(max_epoch):
    sigm1 = predict(theta2, X1)
    loss1 = - np.mean(np.log(sigm1) * Y1 + np.log(1 - sigm1) * (1 - Y1))
    rmsp_loss.append(loss1)
    
    gradient = np.sum(np.dot((sigm1 - Y1.flatten()), X1))/len(sigm1)
    
    e_sq_grad = gamma * e_sq_grad + (1 - gamma)  * gradient ** 2
    
    theta2 -= lr * gradient / np.sqrt(e_sq_grad + eps)
print(theta2)

[ 0.43075345  1.14986186 -0.86131271 -0.47454016 -1.78985269]
